# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import time
from copy import deepcopy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

import ollama

In [2]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

### Parameters

In [3]:
# List of models
ml_models = [
    ('Logistic Regression', LogisticRegression(random_state=42, max_iter=1000)), # 2 sec
    ('Random Forest', RandomForestClassifier(random_state=42)), # 2 min
    ('SVM', SVC(probability=True, random_state=42)), # 30 min
    ('KNN', KNeighborsClassifier()), # 30 sec
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)) # 30 sec
]

In [4]:
# LLMs
models = ['llama3.2:1b', 'llama3.2:3b', 'gemma3:1b', 'gemma3:4b', 'llama3.1', 'dolphin3', 'mistral', 'deepseek-llm']

In [5]:
# Vectorize (Bag of words representation)
vectorizer = CountVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))

In [6]:
# Number of rows that LLM will generate and compare to
n_rows = 5000

In [7]:
# list of all letters to stop on (LLM should only return number 1 to 10
stop_chars = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Load data

In [8]:
# Load the CSV file into a pandas DataFrame
# We can only make a fair comparison if we partition similar train and test set 
# train 4000 --> test 1000
# train llm features 4000 --> 1000 test 
df = pd.read_csv('IMB_preprocessed_2025_04_06.csv')[:n_rows] 

# Display the first 5 records
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. \nThe filming t...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


# LLM Feature Generation

### Qualitative loop

In [9]:
prompt = '''Analyze this movie review and provide a WHOLE NUMBER rating between 1 and 10 for the "{metric}".
Return ONLY a single digit, with no other text, symbols, or explanations.

Review:
{review}'''

In [10]:
# Review metrics
metrics = ['plot', 'characters', 'acting & performances', 'cinematography']

In [14]:
# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # For each row
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Classifying rows"):
        # For each feature that we will generate 
        for metric in metrics:
            # Format the prompt based on the row (review) and the metric
            formatted_prompt = prompt.format(review=row['review'], metric=metric)

            raw_response = None
            attempts = 0
            success = False
            
            # Try up to 10 times
            while attempts < 10 and not success:
                try:
                    # Generate feature 
                    raw_response = ollama.generate(model=model, prompt=formatted_prompt, options={"stop": stop_chars})['response'] # Stop with letters
                    response = int(''.join(filter(str.isdigit, raw_response)))
    
                    # Assert validity response
                    assert 1 <= response <= 10
                    
                    # Store response
                    df.at[index, f"{model}_{metric}"] = response
                    success = True
                    
                except Exception as e:
                    attempts += 1
            
            # If all attempts failed, store 5
            if not success:
                df.at[index, f"{model}_{metric}"] = 5
                print(f"All attempts failed for model {model}, metric {metric}, row {index}. Storing default value 5.")
                print(raw_response)

Processing Model: dolphin3 (Model 1/3)


Classifying rows:  60%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                      | 3024/5000 [3:43:32<1:52:06,  3.40s/it]

All attempts failed for model dolphin3, metric plot, row 3024. Storing default value 5.
I
All attempts failed for model dolphin3, metric characters, row 3024. Storing default value 5.
Do
All attempts failed for model dolphin3, metric acting & performances, row 3024. Storing default value 5.
T


Classifying rows:  60%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                      | 3025/5000 [3:43:45<3:23:03,  6.17s/it]

All attempts failed for model dolphin3, metric cinematography, row 3024. Storing default value 5.
Do


...

Classifying rows:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 4951/5000 [6:37:32<04:26,  5.45s/it]

All attempts failed for model deepseek-llm, metric plot, row 4951. Storing default value 5.
Th


Classifying rows: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 4986/5000 [6:40:14<01:14,  5.35s/it]

All attempts failed for model deepseek-llm, metric cinematography, row 4985. Storing default value 5.
T


Classifying rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [6:41:12<00:00,  4.81s/it]


In [15]:
models = ['llama3.2:1b', 'llama3.2:3b', 'gemma3:1b', 'gemma3:4b', 'llama3.1', 'dolphin3', 'mistral', 'deepseek-llm']

In [16]:
df.head()

,review,sentiment,llama3.2:1b_plot,llama3.2:1b_characters,llama3.2:1b_acting & performances,llama3.2:1b_cinematography,llama3.2:3b_plot,llama3.2:3b_characters,llama3.2:3b_acting & performances,llama3.2:3b_cinematography,...,dolphin3_acting & performances,dolphin3_cinematography,mistral_plot,mistral_characters,mistral_acting & performances,mistral_cinematography,deepseek-llm_plot,deepseek-llm_characters,deepseek-llm_acting & performances,deepseek-llm_cinematography
0,One of the other reviewers has mentioned that ...,1,4.0,1.0,9.0,5.0,9.0,8.0,8.0,8.0,...,9.0,8.0,8.0,8.0,8.0,8.0,7.0,9.0,9.0,7.0
1,A wonderful little production. \nThe filming t...,1,6.0,5.0,8.0,3.0,9.0,8.0,8.0,8.0,...,9.0,8.0,9.0,9.0,9.0,9.0,4.0,4.0,4.0,4.0
2,I thought this was a wonderful way to spend ti...,1,3.0,2.0,1.0,3.0,7.0,8.0,8.0,8.0,...,8.0,7.0,8.0,7.0,8.0,8.0,4.0,4.0,4.0,4.0
3,Basically there's a family where a little boy ...,0,2.0,2.0,2.0,2.0,5.0,7.0,6.0,4.0,...,5.0,6.0,5.0,6.0,6.0,6.0,7.0,4.0,6.0,4.0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,5.0,3.0,6.0,6.0,7.0,8.0,7.0,8.0,...,9.0,9.0,7.0,8.0,8.0,9.0,4.0,4.0,4.0,4.0


# Comparison

### Split the data
We make a train and test split, however the train is seperated in the LLM generated features and the review text

In [17]:
def experimental_setup(data, model):
    # We select the cols review, sentiment and the metric columns for the model (LLM)
    selected_cols = ['review', 'sentiment'] + [f"{model}_{metric}" for metric in metrics]
    df = data.copy()[selected_cols]
    
    # Split into X and y
    X = df.drop('sentiment', axis=1)
    y = df['sentiment']
    
    # Split data into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_text = X_train['review']
    X_train_tab = X_train.drop('review', axis=1)
    X_test_text = X_test['review']
    X_test_tab = X_test.drop('review', axis=1)
    
    # Transform text reviews to Bag of Words representation
    X_train_text = vectorizer.fit_transform(X_train_text)
    X_test_text = vectorizer.transform(X_test_text)
    
    # Print the shapes of the sets
    print(f"Train shapes - Text: {X_train_text.shape}, Tab: {X_train_tab.shape}, Label: {y_train.shape}")
    print(f"Test shapes  - Text: {X_test_text.shape}, Tab: {X_test_tab.shape}, Label: {y_test.shape}")
    print()

    return X_train_text, X_train_tab, y_train, X_test_text, X_test_tab, y_test

### Method to train model and get performance metrics

In [18]:
# Function to train and evaluate models with multiple metrics
def evaluate_models_with_metrics(models, X_train, y_train, X_test, y_test):
    results = []

    for name, model in models:      
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_test_pred = model.predict(X_test)

        # Store results for the model
        model_results = {
            'Model': name,
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Test Precision': precision_score(y_test, y_test_pred),
            'Test Recall': recall_score(y_test, y_test_pred),
            'Test F1-Score': f1_score(y_test, y_test_pred)
        }
        
        results.append(model_results)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results).set_index("Model")
    
    return results_df

## Textual vs LLM Generated Features - Classifcations

In [19]:
# For each LLM
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # Generate the data split
    X_train_text, X_train_tab, y_train, X_test_text, X_test_tab, y_test = experimental_setup(df, model)

    # Evaluate models with multiple metrics and print results
    textual_results = evaluate_models_with_metrics(ml_models, X_train_text, y_train, X_test_text, y_test)
    tabular_results = evaluate_models_with_metrics(ml_models, X_train_tab, y_train, X_test_tab, y_test) 
    
    # Compute absolute and percentage differences
    abs_diff = (textual_results - tabular_results).abs().round(3)
    percent_diff = ((abs_diff / tabular_results) * 100).round(2)
    
    # Display results
    print("Predictions on review text:")
    display(textual_results)
    print("Predictions on tabular features derived from text (by LLM):")
    display(tabular_results)
    print("Absolute Differences:")
    display(abs_diff)
    print("Percentage Differences:")
    display(percent_diff)
    print()

Processing Model: llama3.2:1b (Model 1/8)
Train shapes - Text: (4000, 10000), Tab: (4000, 4), Label: (4000,)
Test shapes  - Text: (1000, 10000), Tab: (1000, 4), Label: (1000,)

Predictions on review text:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.866,0.840081,0.882979,0.860996
Random Forest,0.834,0.833333,0.808511,0.820734
SVM,0.841,0.806706,0.870213,0.837257
KNN,0.586,0.560606,0.551064,0.555794
Gradient Boosting,0.786,0.742424,0.834043,0.785571


Predictions on tabular features derived from text (by LLM):


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.674,0.660714,0.629787,0.644880
Random Forest,0.597,0.571734,0.568085,0.569904
SVM,0.679,0.662309,0.646809,0.654467
KNN,0.635,0.610994,0.614894,0.612937
Gradient Boosting,0.667,0.643606,0.653191,0.648363


Absolute Differences:


...

,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.686,0.681395,0.623404,0.651111
Random Forest,0.838,0.776978,0.919149,0.842105
SVM,0.779,0.704433,0.912766,0.795181
KNN,0.745,0.804533,0.604255,0.690158
Gradient Boosting,0.837,0.771681,0.927660,0.842512


Absolute Differences:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.180,0.159,0.260,0.210
Random Forest,0.004,0.056,0.111,0.021
SVM,0.062,0.102,0.043,0.042
KNN,0.159,0.244,0.053,0.134
Gradient Boosting,0.051,0.029,0.094,0.057


Percentage Differences:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,26.24,23.33,41.71,32.25
Random Forest,0.48,7.21,12.08,2.49
SVM,7.96,14.48,4.71,5.28
KNN,21.34,30.33,8.77,19.42
Gradient Boosting,6.09,3.76,10.13,6.77


## Textual vs Ensemble - Classifcations

In [20]:
def evaluate_ensemble_model(ml_model_textual, ml_model_tabular): 
    assert ml_model_textual[0] == ml_model_tabular[0]
    model_name = ml_model_textual[0]
    
    # Extract model from tuple
    ml_model_tabular = ml_model_tabular[1]
    ml_model_textual = ml_model_textual[1]
    
    # Get prediction probabilities for both modalities       
    probas_tab_test = ml_model_tabular.predict_proba(X_test_tab)
    probas_txt_test = ml_model_textual.predict_proba(X_test_text)

    # Combine probabilities (average them)
    combined_proba_test = (probas_tab_test + probas_txt_test) / 2
    
    # Get final predictions
    y_pred_test = np.argmax(combined_proba_test, axis=1)

    # Store results
    ensemble_results = ({
        'Model': model_name,
        'Test Accuracy': accuracy_score(y_test, y_pred_test),
        'Test Precision': precision_score(y_test, y_pred_test),
        'Test Recall': recall_score(y_test, y_pred_test),
        'Test F1-Score': f1_score(y_test, y_pred_test)
    })

    return ensemble_results

In [21]:
# For each LLM
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # Generate the data split
    X_train_text, X_train_tab, y_train, X_test_text, X_test_tab, y_test = experimental_setup(df, model)

    # Evaluate models with multiple metrics and print results
    # This also has the effect of training the models in ml_models
    textual_results = evaluate_models_with_metrics(ml_models, X_train_text, y_train, X_test_text, y_test)
    textual_models = deepcopy(ml_models) # Save the list of trained models
        
    tabular_results = evaluate_models_with_metrics(ml_models, X_train_tab, y_train, X_test_tab, y_test)
    tabular_models = deepcopy(ml_models) # Save the list of trained models

    # We will store the results of the ensemble method
    ensemble_results_lst = []
    
    # Loop over the tabular and textual ml models to make ensemble models
    for ml_model_tabular, ml_model_textual in zip(tabular_models, textual_models):
        ensemble_results_lst.append(evaluate_ensemble_model(ml_model_textual, ml_model_tabular))

    ensemble_results = pd.DataFrame(ensemble_results_lst).set_index("Model")

    # Calculate percentage difference tables
    text_diff_pct = (100 * (ensemble_results - textual_results) / textual_results).round(2)
    tab_diff_pct = (100 * (ensemble_results - tabular_results) / tabular_results).round(2)
    
    # Display all results
    print("Ensemble Results:")
    display(ensemble_results)
    print("% Difference ensemble vs textual model:")
    display(text_diff_pct)
    print("% Difference ensemble vs tabular model:")
    display(tab_diff_pct)
    print()

Processing Model: llama3.2:1b (Model 1/8)
Train shapes - Text: (4000, 10000), Tab: (4000, 4), Label: (4000,)
Test shapes  - Text: (1000, 10000), Tab: (1000, 4), Label: (1000,)

Ensemble Results:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.877,0.860707,0.880851,0.870662
Random Forest,0.739,0.717256,0.734043,0.725552
SVM,0.856,0.829960,0.872340,0.850622
KNN,0.642,0.652174,0.510638,0.572792
Gradient Boosting,0.813,0.796646,0.808511,0.802534


% Difference ensemble vs textual model:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,1.27,2.46,-0.24,1.12
Random Forest,-11.39,-13.93,-9.21,-11.60
SVM,1.78,2.88,0.24,1.60
KNN,9.56,16.33,-7.34,3.06
Gradient Boosting,3.44,7.30,-3.06,2.16


% Difference ensemble vs tabular model:


...

,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.869,0.850932,0.874468,0.862539
Random Forest,0.868,0.816479,0.927660,0.868526
SVM,0.881,0.846154,0.912766,0.878199
KNN,0.777,0.845938,0.642553,0.730351
Gradient Boosting,0.887,0.838710,0.940426,0.886660


% Difference ensemble vs textual model:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,0.35,1.29,-0.96,0.18
Random Forest,4.08,-2.02,14.74,5.82
SVM,4.76,4.89,4.89,4.89
KNN,32.59,50.90,16.60,31.41
Gradient Boosting,12.85,12.97,12.76,12.87


% Difference ensemble vs tabular model:


,Test Accuracy,Test Precision,Test Recall,Test F1-Score
Model,,,,
Logistic Regression,26.68,24.88,40.27,32.47
Random Forest,3.58,5.08,0.93,3.14
SVM,13.09,20.12,0.00,10.44
KNN,4.30,5.15,6.34,5.82
Gradient Boosting,5.97,8.69,1.38,5.24
